In [ ]:
!pip install transformers torch
!pip install pymupdf Pillow opencv-python-headless
!pip install ipywidgets --upgrade



In [3]:
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Processor
from PIL import Image
import torch

# Load the LayoutLMv3 model
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=5)

# Load the processor
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base")


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def extract_data_from_image(image):
    # Process the image using the processor
    inputs = processor(images=image, return_tensors="pt")
    
    # Forward pass to get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the logits and convert to predicted labels
    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()

    # Decode the tokens and predictions
    tokens = processor.tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    labels = [model.config.id2label[pred] for pred in predictions]

    return tokens, labels

# Example usage for the first page/image
tokens, labels = extract_data_from_image(images[0])

# Print the tokens with their labels
for token, label in zip(tokens, labels):
    print(f"{token}: {label}")
